In [ ]:
# import python packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle
import os
import importlib
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import itertools
from scipy import stats

In [ ]:
site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
ext_list = ['csv','dsv', 'dsv', 'csv', 'csv', 'csv', 'csv', 'csv', 'csv']
sep_list = [',','|', '|', '|', ',', ',', ',', ',', '|']
encoding_list = ['utf-8','utf-8','utf-8','utf-8','utf-8','utf-8', 'windows-1252', 'utf-8', 'utf-16'] 

In [ ]:
for ct in range(len(site_list)):

    site = site_list[ct]
    ext = ext_list[ct]
    sep = sep_list[ct]
    encoding = encoding_list[ct]
    path = []
    print(site)

    if site != 'KUMC':
        rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '/raw/'
    else: 
        rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '_ORCALE/raw/'
    path.append(rawpath)
    path.append('/blue/yonghui.wu/hoyinchan/Data/data2022/' + site + '/')
    pdata = '/blue/yonghui.wu/hoyinchan/Data/data2022/'+ site 
    site

    def try_load_csv(dataname, site, ext, sep, path, datecol=None):

        print(f"processing : {dataname}")
        filename = 'AKI_'+dataname.upper()
        outfilename = f"/p0_{dataname}_{site}.parquet"    

        # Site Specific filenames
        if 'UMHC' in path[0]:
            filename = 'DEID_'+filename                  

        try:
            if site == 'KUMC':
                df = pd.read_csv(path[0] +  filename + '.' + ext, sep=sep, encoding=encoding)
            else:
                df = pd.read_csv(path[0] +  filename + '.' + ext, sep=sep, encoding=encoding, engine='python', on_bad_lines='skip')

        except Exception as e:
            print(f"{site}: {dataname} Failed to load from {path[0]} as well: {e}")

        df.rename(columns = {'\ufeff"ONSETS_ENCOUNTERID"': 'ONSETS_ENCOUNTERID'}, inplace = True) # to handle the BOM character in UTHSCSA
        df.columns = df.columns.str.upper()
        df.columns = df.columns.str.replace('"+PD.DATE_SHIFT"','').str.replace('AKI.','') # To handle the starnge date name in KUMC
        df['PATID'] = df['PATID'].astype(str)
        df['ENCOUNTERID'] = df['ENCOUNTERID'].astype(str)
        df['BCCOVID'] = df['BCCOVID'].astype(bool)
        return df

    onsets = try_load_csv('onsets', site, ext, sep, path, datecol=None)

    onsets = onsets[['PATID', 'ENCOUNTERID', 'BCCOVID']]

    outfilename = f"/p0_covid_status_{site}.parquet"
    onsets.to_parquet(pdata+outfilename)

In [ ]:
for site in site_list:
    
    onsets = pd.read_parquet(pdata+outfilename)
    print(onsets)